<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#RedShift-Notebook" data-toc-modified-id="RedShift-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RedShift Notebook</a></span></li><li><span><a href="#Create-Role" data-toc-modified-id="Create-Role-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create Role</a></span></li><li><span><a href="#Create-Cluster" data-toc-modified-id="Create-Cluster-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create Cluster</a></span></li><li><span><a href="#DO-NOT-RUN-CELL-BELOW-UNLESS-ClusterStatus-=-'Available'" data-toc-modified-id="DO-NOT-RUN-CELL-BELOW-UNLESS-ClusterStatus-=-'Available'-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DO NOT RUN CELL BELOW UNLESS ClusterStatus = 'Available'</a></span></li><li><span><a href="#Copy-and-Paste-Endpoint-and-ARN-into-dwh.cfg-file" data-toc-modified-id="Copy-and-Paste-Endpoint-and-ARN-into-dwh.cfg-file-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Copy and Paste Endpoint and ARN into dwh.cfg file</a></span></li><li><span><a href="#Open-TCP-Port" data-toc-modified-id="Open-TCP-Port-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Open TCP Port</a></span></li><li><span><a href="#Validate-ETL-Pipeline" data-toc-modified-id="Validate-ETL-Pipeline-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Validate ETL Pipeline</a></span><ul class="toc-item"><li><span><a href="#Staging-Tables" data-toc-modified-id="Staging-Tables-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Staging Tables</a></span><ul class="toc-item"><li><span><a href="#Staging-Events-Table" data-toc-modified-id="Staging-Events-Table-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Staging Events Table</a></span></li><li><span><a href="#Staging-Songs-Table" data-toc-modified-id="Staging-Songs-Table-7.1.2"><span class="toc-item-num">7.1.2&nbsp;&nbsp;</span>Staging Songs Table</a></span></li></ul></li><li><span><a href="#Fact-and-Dimension-Tables" data-toc-modified-id="Fact-and-Dimension-Tables-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Fact and Dimension Tables</a></span><ul class="toc-item"><li><span><a href="#Dimension-Table:-Time" data-toc-modified-id="Dimension-Table:-Time-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Dimension Table: Time</a></span></li><li><span><a href="#Dimension-Table:-Artists" data-toc-modified-id="Dimension-Table:-Artists-7.2.2"><span class="toc-item-num">7.2.2&nbsp;&nbsp;</span>Dimension Table: Artists</a></span></li><li><span><a href="#Dimension-Table:-Songs" data-toc-modified-id="Dimension-Table:-Songs-7.2.3"><span class="toc-item-num">7.2.3&nbsp;&nbsp;</span>Dimension Table: Songs</a></span></li><li><span><a href="#Dimension-Table:-Users" data-toc-modified-id="Dimension-Table:-Users-7.2.4"><span class="toc-item-num">7.2.4&nbsp;&nbsp;</span>Dimension Table: Users</a></span></li><li><span><a href="#Fact-Table:-Songplay" data-toc-modified-id="Fact-Table:-Songplay-7.2.5"><span class="toc-item-num">7.2.5&nbsp;&nbsp;</span>Fact Table: Songplay</a></span></li></ul></li></ul></li><li><span><a href="#Delete-Cluster-to-Elimate-Unnecessary-AWS-Charges" data-toc-modified-id="Delete-Cluster-to-Elimate-Unnecessary-AWS-Charges-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Delete Cluster to Elimate Unnecessary AWS Charges</a></span></li><li><span><a href="#Detach-and-Delete-Role" data-toc-modified-id="Detach-and-Delete-Role-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Detach and Delete Role</a></span></li></ul></div>

# RedShift Notebook

Notebook Purpose:
- Initiate RedShift cluster and role (with attached policy)
- Validate table creation (create_tables.py) and ETL pipeline (etl.py)
- Delete RedShift cluster and role

In [ ]:
import pandas as pd
import boto3
import json

The cell below extracts values from the dwh.cfg file to be used for RedShift cluster and Postgres database configuration

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","DWH_CLUSTER_IDENTIFIER")
DWH_DB_NAME                 = config.get("CLUSTER","DWH_DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DWH_DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DWH_DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("CLUSTER", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB_NAME)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB_NAME", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB_NAME, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

In [ ]:
#initiate aws clients

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [ ]:
# access udacity's s3 bucket used for data extraction
bucket = s3.Bucket('udacity-dend')

In [ ]:
# sample paths to log json files used for extraction
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='log-data')]
log_data_files[:10]

In [ ]:
# sample paths to song json files used for extraction
song_data_files = [filename.key for filename in bucket.objects.filter(Prefix='song-data/A/A')]
song_data_files[:10]

# Create Role

In [ ]:
#creates role for redshift cluster
try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {
                'Statement': [{'Action': 'sts:AssumeRole',
                               'Effect': 'Allow',
                               'Principal': {'Service': 'redshift.amazonaws.com'}}],
                'Version': '2012-10-17'
            })
    )
except Exception as e:
    print(e)

In [ ]:
#attaches s3 read only policy to role
iam.attach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")['ResponseMetadata']['HTTPStatusCode']

In [ ]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

# Create Cluster

In [ ]:
try:
    response = redshift.create_cluster(        
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [ ]:
def prettyRedshiftProps(props):
    """Displays exclusive RedShift cluster information in a pretty pandas dataframe
    
    Arg:
    props -- All RedShift cluster information
    
    """
    
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

# DO NOT RUN CELL BELOW UNLESS ClusterStatus = 'Available'
If do, then recreate cluster

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']

# Copy and Paste Endpoint and ARN into dwh.cfg file

In [ ]:
print('Endpoint: ', DWH_ENDPOINT)
print('ARN: ', DWH_ROLE_ARN)

# Open TCP Port

In [ ]:
#open tcp port to access cluster endpoint

try:
    #virtual private cloud where cluster is located
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    #default security groups from vpc
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    #authorize traffic from any IP using TCP from specified port
    defaultSg.authorize_ingress(
        GroupName='default',
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

# Validate ETL Pipeline

In [ ]:
#sql magic to execute sql queries cells
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB_NAME)
print(conn_string)
%sql $conn_string

## Staging Tables

The below tables are samples of the staging events and songs tables. 

### Staging Events Table

In [ ]:
%sql select * FROM staging_events LIMIT 5;

### Staging Songs Table

In [ ]:
%sql select * FROM staging_songs LIMIT 5;

## Fact and Dimension Tables

The tables below are samples of the fact and dimension tables and their respective table count. 

### Dimension Table: Time

In [ ]:
%sql SELECT COUNT(*) FROM dim_time;

In [ ]:
%sql SELECT * FROM dim_time LIMIT 5;

### Dimension Table: Artists


In [ ]:
%sql SELECT COUNT(*) FROM dim_artists;

In [ ]:
%sql SELECT * FROM dim_artists LIMIT 5;

### Dimension Table: Songs


In [ ]:
%sql SELECT COUNT(*) FROM dim_songs;

In [ ]:
%sql SELECT * FROM dim_songs LIMIT 5;

### Dimension Table: Users


In [ ]:
%sql SELECT COUNT(*) FROM dim_users;

In [ ]:
%sql SELECT * FROM dim_users LIMIT 5;

### Fact Table: Songplay


In [ ]:
%sql SELECT COUNT(*) FROM fact_songplay;

In [ ]:
%sql SELECT * FROM fact_songplay LIMIT 5;

# Delete Cluster to Elimate Unnecessary AWS Charges

In [ ]:
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)

In [ ]:
try:
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
except Exception as e:
    print('Cluster Deleted')
    print('\nError: ', e)

prettyRedshiftProps(myClusterProps)

# Detach and Delete Role

In [ ]:
try:
    iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
    iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
except Exception as e:
    print('IAM Role Deleted')
    print('\nError: ', e)